In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

Pour utiliser sur google colab et profiter du GPU

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Lecture des fichiers 

In [ ]:
data_dir = '/content/drive/MyDrive/DATASET/machine_translation/'

In [ ]:
data_path_fr = data_dir+'small_vocab_fr.txt'
data_path_en = data_dir+'small_vocab_en.txt'

On  lit les fichiers et on les stocke dans des listes

In [ ]:
text_fr = open(data_path_fr,'r',encoding='utf8').read()
text_en = open(data_path_en,'r',encoding='utf8').read()

On affiche les 1000 premiers mot 

In [ ]:
text_fr[:1000]

"new jersey est parfois calme pendant l' automne , et il est neigeux en avril .\nles états-unis est généralement froid en juillet , et il gèle habituellement en novembre .\ncalifornia est généralement calme en mars , et il est généralement chaud en juin .\nles états-unis est parfois légère en juin , et il fait froid en septembre .\nvotre moins aimé fruit est le raisin , mais mon moins aimé est la pomme .\nson fruit préféré est l'orange , mais mon préféré est le raisin .\nparis est relaxant en décembre , mais il est généralement froid en juillet .\nnew jersey est occupé au printemps , et il est jamais chaude en mars .\nnotre fruit est moins aimé le citron , mais mon moins aimé est le raisin .\nles états-unis est parfois occupé en janvier , et il est parfois chaud en novembre .\nla chaux est son moins aimé des fruits , mais la banane est mon moins aimé.\nil a vu un vieux camion jaune .\ninde est pluvieux en juin , et il est parfois chaud en novembre .\nce chat était mon animal le plus ai

Creation de deux nouvelles listes, qui contiennet le text divisé ligne par ligne

In [ ]:
lines_fr = text_fr.split('\n')
lines_en = text_en.split('\n')

Affichage des 10 premieres lignes en francais

In [ ]:
lines_fr[:10]

["new jersey est parfois calme pendant l' automne , et il est neigeux en avril .",
 'les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .',
 'california est généralement calme en mars , et il est généralement chaud en juin .',
 'les états-unis est parfois légère en juin , et il fait froid en septembre .',
 'votre moins aimé fruit est le raisin , mais mon moins aimé est la pomme .',
 "son fruit préféré est l'orange , mais mon préféré est le raisin .",
 'paris est relaxant en décembre , mais il est généralement froid en juillet .',
 'new jersey est occupé au printemps , et il est jamais chaude en mars .',
 'notre fruit est moins aimé le citron , mais mon moins aimé est le raisin .',
 'les états-unis est parfois occupé en janvier , et il est parfois chaud en novembre .']

Affichage des 10 premieres lignes en anglais

In [ ]:
lines_en[:10]

['new jersey is sometimes quiet during autumn , and it is snowy in april .',
 'the united states is usually chilly during july , and it is usually freezing in november .',
 'california is usually quiet during march , and it is usually hot in june .',
 'the united states is sometimes mild during june , and it is cold in september .',
 'your least liked fruit is the grape , but my least liked is the apple .',
 'his favorite fruit is the orange , but my favorite is the grape .',
 'paris is relaxing during december , but it is usually chilly in july .',
 'new jersey is busy during spring , and it is never hot in march .',
 'our least liked fruit is the lemon , but my least liked is the grape .',
 'the united states is sometimes busy during january , and it is sometimes warm in november .']

Affichage simultané des 5 premieres lignes en anglais avec leur traduction en francais 

In [ ]:
for i in range(5):
    print(i, ' english sentence : ', lines_en[i])
    print(i, ' french sentence : ', lines_fr[i])
    print('\n')

0  english sentence :  new jersey is sometimes quiet during autumn , and it is snowy in april .
0  french sentence :  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .


1  english sentence :  the united states is usually chilly during july , and it is usually freezing in november .
1  french sentence :  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


2  english sentence :  california is usually quiet during march , and it is usually hot in june .
2  french sentence :  california est généralement calme en mars , et il est généralement chaud en juin .


3  english sentence :  the united states is sometimes mild during june , and it is cold in september .
3  french sentence :  les états-unis est parfois légère en juin , et il fait froid en septembre .


4  english sentence :  your least liked fruit is the grape , but my least liked is the apple .
4  french sentence :  votre moins aimé fruit est le raisin , mais mon

In [ ]:
total_word_fr = text_fr.split(' ')
total_word_en = text_en.split(' ')
print('nombre de mots fr :',len(total_word_fr))
print('nombre de mots en : ', len(total_word_en))

nombre de mots fr : 1823439
nombre de mots en :  1685433


# Tokenization

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

La tokenization est l'etape qui consiste a associer a chaque mot contenu dans notre dataset, un nombre.


La fonction suivante, prend en entrée un texte, et va renvoyer dans un premier temps le tokenizer, c est a dire l'outil qui permet d associer a chaque mot un nombre, et le texte encodé, c est a dire une sequence ou chaque mot y est remplacé par son nombre associé 

In [ ]:
def tokenize(x):
    tokenizer=Tokenizer(filters='!"#$%&()*+-/:;<=>?@[\\]^_`{|}~\t\n')
    tokenizer.fit_on_texts(x)
    return tokenizer, tokenizer.texts_to_sequences(x)

il faut creer un tokenizer en francais et en anglais puis creer les sequences avec ce tokenizer

In [ ]:
tok_en, sequences_en = tokenize(lines_en)

In [ ]:
sequences_en[0]

[19, 25, 1, 10, 69, 6, 41, 2, 9, 5, 1, 57, 4, 46, 3]

In [ ]:
tok_fr, sequences_fr = tokenize(lines_fr)

Le word_index contien l'association du mot et de son code 

In [ ]:
tok_en.word_index

In [ ]:
tok_fr.word_index

On affiche les 5 premieres lignes en anglais et en francais ainsi que leur sequence associée 

In [ ]:
for i in range(5):
    print(i, ' english sentence : ', lines_en[i])
    print(i, ' english sequence : ', sequences_en[i])
    print(i, ' french sentence : ', lines_fr[i])
    print(i, ' french sequence : ', sequences_fr[i])
    
    
    print('\n')

# Padding des sequences

Le padding consiste a rendre la taille de toutes les sequences identiques en rajoutant des 0 a la fin

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
max_len_sequence_en = max([len(x) for x in sequences_en])
print('len max en :',max_len_sequence_en)
max_len_sequence_fr = max([len(x) for x in sequences_fr])
print('len max fr :',max_len_sequence_fr)

len max en : 17
len max fr : 23


In [ ]:
def padding(x,length):
    padded = pad_sequences(x,maxlen = length, padding='post')
    return padded

In [ ]:
padded = padding(sequences_en,None)
padded_fr = padding(sequences_fr,None)

In [ ]:
for i in range(10):
    print(i,' original sequence : ', sequences_en[i])
    print(i, ' padded sequence :   ',padded[i])
    print('\n')

0  original sequence :  [19, 25, 1, 10, 69, 6, 41, 2, 9, 5, 1, 57, 4, 46, 3]
0  padded sequence :    [19 25  1 10 69  6 41  2  9  5  1 57  4 46  3  0  0]


1  original sequence :  [7, 22, 23, 1, 11, 64, 6, 45, 2, 9, 5, 1, 11, 53, 4, 47, 3]
1  padded sequence :    [ 7 22 23  1 11 64  6 45  2  9  5  1 11 53  4 47  3]


2  original sequence :  [24, 1, 11, 69, 6, 40, 2, 9, 5, 1, 11, 70, 4, 36, 3]
2  padded sequence :    [24  1 11 69  6 40  2  9  5  1 11 70  4 36  3  0  0]


3  original sequence :  [7, 22, 23, 1, 10, 66, 6, 36, 2, 9, 5, 1, 59, 4, 44, 3]
3  padded sequence :    [ 7 22 23  1 10 66  6 36  2  9  5  1 59  4 44  3  0]


4  original sequence :  [31, 13, 18, 15, 1, 7, 85, 2, 8, 32, 13, 18, 1, 7, 87, 3]
4  padded sequence :    [31 13 18 15  1  7 85  2  8 32 13 18  1  7 87  3  0]


5  original sequence :  [33, 14, 15, 1, 7, 93, 2, 8, 32, 14, 1, 7, 85, 3]
5  padded sequence :    [33 14 15  1  7 93  2  8 32 14  1  7 85  3  0  0  0]


6  original sequence :  [20, 1, 68, 6, 49, 2, 8, 5, 

In [ ]:
for i in range(10):
    print(i,' original sequence : ', sequences_fr[i])
    print(i, ' padded sequence :   ',padded_fr[i])
    print('\n')

0  original sequence :  [37, 36, 1, 10, 69, 39, 13, 26, 3, 8, 5, 1, 114, 4, 52, 2]
0  padded sequence :    [ 37  36   1  10  69  39  13  26   3   8   5   1 114   4  52   2   0   0
   0   0   0   0   0]


1  original sequence :  [6, 34, 33, 1, 14, 21, 4, 51, 3, 8, 5, 97, 71, 4, 53, 2]
1  padded sequence :    [ 6 34 33  1 14 21  4 51  3  8  5 97 71  4 53  2  0  0  0  0  0  0  0]


2  original sequence :  [103, 1, 14, 69, 4, 47, 3, 8, 5, 1, 14, 23, 4, 43, 2]
2  padded sequence :    [103   1  14  69   4  47   3   8   5   1  14  23   4  43   2   0   0   0
   0   0   0   0   0]


3  original sequence :  [6, 34, 33, 1, 10, 273, 4, 43, 3, 8, 5, 105, 21, 4, 50, 2]
3  padded sequence :    [  6  34  33   1  10 273   4  43   3   8   5 105  21   4  50   2   0   0
   0   0   0   0   0]


4  original sequence :  [42, 15, 17, 18, 1, 12, 84, 3, 7, 41, 15, 17, 1, 9, 85, 2]
4  padded sequence :    [42 15 17 18  1 12 84  3  7 41 15 17  1  9 85  2  0  0  0  0  0  0  0]


5  original sequence :  [22, 18, 19

# La pipeline

In [ ]:
def preprocess(x,y):
    x_tok, sequence_x = tokenize(x)
    y_tok, sequence_y = tokenize(y)
    
    padded_x = padding(sequence_x,None)
    padded_y = padding(sequence_y,None)
    
    padded_y = np.expand_dims(padded_y, axis=-1)
    
    return(padded_x, padded_y, x_tok, y_tok)

In [ ]:
padded_en, padded_fr, tok_en, tok_fr = preprocess(lines_en,lines_fr)

In [ ]:
padded_fr.shape

(137861, 23, 1)

In [ ]:
padded_en

array([[19, 25,  1, ...,  3,  0,  0],
       [ 7, 22, 23, ...,  4, 47,  3],
       [24,  1, 11, ...,  3,  0,  0],
       ...,
       [26,  1, 12, ...,  3,  0,  0],
       [ 7, 93,  1, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int32)

# Conversion one  hot encodinng

format des données en on hot encoding : x ( len(x), taille de la sequence paddé, longueur vocab x )

y ( len(y), taille de la sequence paddé, longueur vocab y )


In [ ]:
print(len(lines_fr))
print(len(padded_fr))
print(padded_fr.shape)
padded_en.shape


137861
137861


(137861, 17)

In [ ]:
#x_ohe_data = np.zeros((padded_en.shape[0],padded_en.shape[1],len(tok_en.word_index)))
#y_ohe_data= np.zeros((padded_fr.shape[0],padded_fr.shape[1],len(tok_fr.word_index)))

In [ ]:
#print(x_ohe_data.shape)
#print(y_ohe_data.shape)

# Decodage

In [ ]:
print("max len sequence en :",padded_en.shape[1])
print("max len sequence fr :",padded_fr.shape[1])
print("number words en :", len(tok_en.word_index))
print("number words fr :", len(tok_fr.word_index))
print("number sequences en :", len(padded_en))
print("number sequences fr :", len(padded_fr))

max len sequence en : 17
max len sequence fr : 23
number words en : 226
number words fr : 348
number sequences en : 137861
number sequences fr : 137861


In [ ]:
def sequence_to_sentence(sequence, tokenizer):
    decoded_sequence = []
    for i in sequence:
        for word, index in tokenizer.word_index.items():
                if i == index:
                    decoded_sequence.append(word)
                    decoded_sequence.append(' ')
    return ''.join(decoded_sequence)


In [ ]:
padded_en[10]

array([ 7, 86,  1, 34, 13, 18, 15,  2,  8,  7, 89,  1, 32, 13, 18,  3,  0],
      dtype=int32)

In [ ]:
sentence = sequence_to_sentence(padded_en[10], tok_en)
sentence

'the lime is her least liked fruit , but the banana is my least liked . '

# Creation du modele

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, TimeDistributed, LSTM, Bidirectional, Embedding, RepeatVector
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [ ]:
model = Sequential()
model.add(Embedding(len(tok_en.word_index)+1,256))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(tok_fr.word_index)+1, activation='softmax'))
model.compile(loss=sparse_categorical_crossentropy, optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 256)         58112     
_________________________________________________________________
bidirectional (Bidirectional (None, None, 512)         1050624   
_________________________________________________________________
dense (Dense)                (None, None, 1024)        525312    
_________________________________________________________________
dropout (Dropout)            (None, None, 1024)        0         
_________________________________________________________________
dense_1 (Dense)              (None, None, 349)         357725    
Total params: 1,991,773
Trainable params: 1,991,773
Non-trainable params: 0
_________________________________________________________________


In [ ]:
tmp_x = padding(padded_en,padded_fr.shape[1])
print(tmp_x.shape)
#tmp_x = np.expand_dims(tmp_x, axis=-1)
print(tmp_x.shape)
padded_fr = np.squeeze(padded_fr, axis = -1)

(137861, 23)
(137861, 23)


In [ ]:
model.fit(tmp_x,padded_fr,batch_size = 1024, epochs=20, validation_split=0.2)

Epoch 1/20
108/108 [==============================] - 27s 234ms/step - loss: 2.4166 - accuracy: 0.4909 - val_loss: 1.4866 - val_accuracy: 0.6118
Epoch 2/20
108/108 [==============================] - 24s 225ms/step - loss: 1.1801 - accuracy: 0.6792 - val_loss: 0.8448 - val_accuracy: 0.7661
Epoch 3/20
108/108 [==============================] - 24s 223ms/step - loss: 0.7309 - accuracy: 0.7900 - val_loss: 0.5349 - val_accuracy: 0.8396
Epoch 4/20
108/108 [==============================] - 24s 223ms/step - loss: 0.5156 - accuracy: 0.8440 - val_loss: 0.3989 - val_accuracy: 0.8767
Epoch 5/20
108/108 [==============================] - 24s 222ms/step - loss: 0.4046 - accuracy: 0.8734 - val_loss: 0.3295 - val_accuracy: 0.8946
Epoch 6/20
108/108 [==============================] - 24s 222ms/step - loss: 0.3324 - accuracy: 0.8942 - val_loss: 0.2699 - val_accuracy: 0.9132
Epoch 7/20
108/108 [==============================] - 24s 222ms/step - loss: 0.2836 - accuracy: 0.9090 - val_loss: 0.2380 - val_ac

In [ ]:
#simple_rnn_model.save('machine_translation.h5')

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
#model = load_model('machine_translation.h5')

# Test 

In [ ]:
for i in range(10):
  true_translation = lines_fr[i]
  original_text = lines_en[i]
  print(original_text)
  print(true_translation)

  new_tmp = np.expand_dims(tmp_x[i],axis=0)
  #print(new_tmp.shape)
  prediction = model.predict(new_tmp)
  prediction = np.squeeze(prediction, axis=0)
  pred_sequence = []
  for i in range(len(prediction)):
      pred_sequence.append(np.argmax(prediction[i]))
  true_pred = sequence_to_sentence(pred_sequence, tok_fr)
  print(true_pred)
  print("\n")






new jersey is sometimes quiet during autumn , and it is snowy in april .
new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
new jersey est parfois calme pendant l' automne , et il est neigeux en avril . 


the united states is usually chilly during july , and it is usually freezing in november .
les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .
les états unis est généralement froid en juillet , et il gèle habituellement en novembre . 


california is usually quiet during march , and it is usually hot in june .
california est généralement calme en mars , et il est généralement chaud en juin .
california est généralement calme en mars , et il est généralement chaud en juin . 


the united states is sometimes mild during june , and it is cold in september .
les états-unis est parfois légère en juin , et il fait froid en septembre .
les états unis est parfois doux en juin , et il fait froid en septembre . 


your least li

# Test sur des phrases

In [ ]:
test = open('test.txt','r',encoding='utf8').read()
lines_test=test.split('\n')
custom_sequence = tok_en.texts_to_sequences(lines_test)
padded_test= padding(custom_sequence,23)
padded_test = np.expand_dims(padded_test, axis=-1)



In [ ]:
  prediction = model_lstm.predict(padded_test)
  prediction = np.squeeze(prediction, axis=0)
  pred_sequence = []
  for i in range(len(prediction)):
      pred_sequence.append(np.argmax(prediction[i]))
  true_pred = sequence_to_sentence(pred_sequence, tok_fr)
  print(true_pred)
  print("\n")